### 1. 패키지 import

In [1]:
# 기본 패키지
import pandas as pd
import numpy as np
import datetime

# 디렉토리 관련 패키지
import os
import glob

# 시각화 관련 패키지
import seaborn as sns

In [2]:
df_DT = pd.read_csv(r'C:\localRepository\Bigdata_busan\ML\res\working_df.csv', index_col = 0)
df_DT

,Date,Time,Lot,pH,Temp,Voltage,sep
0,2021-09-06,16:29:54,1,2.15,43.15,19.74,1
1,2021-09-06,16:29:59,1,2.08,40.13,18.01,1
2,2021-09-06,16:30:04,1,2.18,43.46,18.73,1
3,2021-09-06,16:30:09,1,1.99,41.72,16.75,1
4,2021-09-06,16:30:14,1,1.85,43.65,18.02,1
...,...,...,...,...,...,...,...
50089,2021-10-27,18:36:03,22,2.05,42.84,15.38,1
50090,2021-10-27,18:36:08,22,1.91,42.64,19.08,1
50091,2021-10-27,18:36:13,22,2.11,44.09,18.14,1
50092,2021-10-27,18:36:18,22,1.92,43.95,17.96,1
